In [1]:
import pandas as pd, re, json, os, boto3, time
from botocore.client import BaseClient
from botocore.exceptions import ClientError as AwsClientError
from utils.utils import s3_folder_cleaner, athena_sql_executor


In [ ]:
"""
bronze (partition):
- create external table if not exists ...
- alter table ... add partition (date=date(...)) location 's3://...'
silver (partition):
- create external table if not exists ...
- insert into table... select...
gold (partition only on fact):
- delete table if exists ...
- create external table ...
- insert overwrite into
"""

In [ ]:
def insert_dim_specs():
    tables=[('')]
    resp = athena_sql_executor(
        "select column_name, data_type "
        "from information_schema.columns "
        "where table_schema = 'c2dwh_silver'and table_name = 'laptops'"
    )

    cols = [
        (i["column_name"], i["data_type"])
        for i in resp.get("data", [])
        if i["column_name"]
        not in [
            "sku",
            "name",
            "price",
            "brand",
            "category",
            "rating",
            "reviews_count",
            "release_year",
            "release_month",
            "url",
            "updated_at",
            "date",
        ]
    ]
    query = (
        "insert into dim_specs with temp as(\n"
        + "\nunion\n".join(
            [
                "select sku product_sku, "
                + f"'{i[0]}' specs_name, "
                + f"{i[0] if i[1]!='varchar' else 'null'} numeric_value, "
                + f"{i[0] if i[1]=='varchar'else 'null'} string_value\n"
                + "from c2dwh_silver.laptops\n"
                + "where date = date('2025-09-09')"
                for i in cols
            ]
        )
        + ")\n"
        + "select row_number()over(order by product_sku) id, product_sku, specs_name, numeric_value, string_value\n"
        + "from temp"
    )

    athena_sql_executor(query, database="c2dwh_gold")
# print(query)

In [ ]:
# athena_sql_executor('drop table dim_specs',database='c2dwh_gold')
# s3_folder_cleaner("gold/dim_specs/", bucket="crawling-to-dwh")